# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_columns', None)

In [2]:
f = open("training.1600000.processed.noemoticon.csv", "r")
print(f.readline())

"0","1467810369","Mon Apr 06 22:19:45 PDT 2009","NO_QUERY","_TheSpecialOne_","@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"



In [3]:
lines = []

for x in f:
    lines.append(x)

In [4]:
len(lines)

1599999

In [5]:
less_lines = []

for i in range(1,1600000,80):
    less_lines.append(lines[i])

In [6]:
len(less_lines)

20000

In [7]:
less_lines[-1]

'"4","2193575955","Tue Jun 16 08:38:42 PDT 2009","NO_QUERY","misscaroline16","scrubbed into surgery today "\n'

In [8]:
less_lines[0].split(',')[4]

'"mattycus"'

In [9]:
re.sub('"', '',less_lines[0].split(',')[4])

'mattycus'

In [11]:
target = []
df_id = []
date = []
flag = []
user = []
text = []

for i in range(len(less_lines)):
    target.append(re.sub('"', '', less_lines[i].split(',')[0]))
    df_id.append(re.sub('"', '', less_lines[i].split(',')[1]))
    date.append(re.sub('"', '', less_lines[i].split(',')[2]))
    flag.append(re.sub('"', '', less_lines[i].split(',')[3]))
    user.append(re.sub('"', '', less_lines[i].split(',')[4]))
    text.append(re.sub('"', '', less_lines[i].split(',')[5]))

In [ ]:
df['target'] = target
df['id'] = df_id
df['date'] = date
df['flag'] = flag
df['user'] = user
df['text'] = text

In [15]:
df

,target,id,date,flag,user,text
0,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
1,0,1467834239,Mon Apr 06 22:25:53 PDT 2009,NO_QUERY,mscha,missin' the boo \n
2,0,1467853479,Mon Apr 06 22:30:56 PDT 2009,NO_QUERY,ringleaderkanon,I feel bad for doing it \n
3,0,1467873256,Mon Apr 06 22:36:06 PDT 2009,NO_QUERY,nicci718,@kristencampisi is this it 4 u? Its ' official...
4,0,1467894749,Mon Apr 06 22:41:54 PDT 2009,NO_QUERY,Fudgey84,wishing i was home underneath my covers \n
...,...,...,...,...,...,...
19995,4,2193477911,Tue Jun 16 08:30:44 PDT 2009,NO_QUERY,Liza_L,@JoAnneJoyM hi !!!! Thank you ! I missed my tw...
19996,4,2193503503,Tue Jun 16 08:32:48 PDT 2009,NO_QUERY,oscarbarber,@perequintana ara sÃ­ que ets tot un &quot;pir...
19997,4,2193527876,Tue Jun 16 08:34:47 PDT 2009,NO_QUERY,MKeithHarris,@denniswords good times indeed \n
19998,4,2193552062,Tue Jun 16 08:36:44 PDT 2009,NO_QUERY,_Jackielynn_,just woke up


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [23]:
# your code here
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords

In [18]:
def clean_up(s):
    # clean URL
    text1 = s.split('http')[0]
    
    #make lower case
    text1 = text1.lower()
    
    # select elements we don't want
    de1 = re.findall('\W', text1)
    de2 = re.findall('\d',text1)
    delete = de1+de2
    
    #deleting elements from text
    for i in delete:
        text1 = text1.replace(i,' ')

    
    return text1

In [19]:
def tokenize(s):
    return word_tokenize(s)

In [20]:
def stem_and_lemmatize(l):
    
    #stemmed
    ps = PorterStemmer()
    l = [ps.stem(w) for w in l]
    
    #lemmatized
    lem = WordNetLemmatizer() 
    l = [lem.lemmatize(word) for word in l]
    
    return l

In [21]:
def remove_stopwords(l):
    return [word for word in l if not word in stopwords.words()]

In [24]:
text_processed = []

for row in df['text']:
    clean = clean_up(row)
    token = tokenize(clean)
    stem_lem = stem_and_lemmatize(token)
    no_stop = remove_stopwords(stem_lem)
    text_processed.append(no_stop)

In [25]:
df['text_processed'] = text_processed

In [26]:
df

,target,id,date,flag,user,text,text_processed
0,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"[kenichan, dive, time, ball, manag, save, rest..."
1,0,1467834239,Mon Apr 06 22:25:53 PDT 2009,NO_QUERY,mscha,missin' the boo \n,"[missin, boo]"
2,0,1467853479,Mon Apr 06 22:30:56 PDT 2009,NO_QUERY,ringleaderkanon,I feel bad for doing it \n,[feel]
3,0,1467873256,Mon Apr 06 22:36:06 PDT 2009,NO_QUERY,nicci718,@kristencampisi is this it 4 u? Its ' official...,"[kristencampisi, offici, round]"
4,0,1467894749,Mon Apr 06 22:41:54 PDT 2009,NO_QUERY,Fudgey84,wishing i was home underneath my covers \n,"[wish, wa, home, underneath, cover]"
...,...,...,...,...,...,...,...
19995,4,2193477911,Tue Jun 16 08:30:44 PDT 2009,NO_QUERY,Liza_L,@JoAnneJoyM hi !!!! Thank you ! I missed my tw...,"[joannejoym, miss, twitter, friend]"
19996,4,2193503503,Tue Jun 16 08:32:48 PDT 2009,NO_QUERY,oscarbarber,@perequintana ara sÃ­ que ets tot un &quot;pir...,"[perequintana, ara, sã, quot, pirata, quot, th..."
19997,4,2193527876,Tue Jun 16 08:34:47 PDT 2009,NO_QUERY,MKeithHarris,@denniswords good times indeed \n,"[dennisword, time, inde]"
19998,4,2193552062,Tue Jun 16 08:36:44 PDT 2009,NO_QUERY,_Jackielynn_,just woke up,[woke]


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [27]:
# your code here
from sklearn.feature_extraction.text import CountVectorizer

In [29]:
bow_vect = CountVectorizer(max_features=5000)
bow_vect
# X = bow_vect.fit_transform(df['text_processed']).toarray()

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [ ]:
# your code here

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [ ]:
# your code here

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here